# Vehicle Model Predictive Control by python

>zhangqq  
>Apr 14, 2023  
>Chongqing
---

<p align=center>
<img src=../img/MPC_vehicleDynamic.png>
</p>

## Kinematic Model
车辆运动学模型:
$$
\begin{bmatrix}
    \dot{X}_r \\
    \dot{Y}_r \\
    \dot{\varphi} \\
\end{bmatrix}
=
\begin{bmatrix}
    \cos \varphi \\
    \sin \varphi \\
    \tan{\delta}/L \\
\end{bmatrix} v_r
$$

该模型满足MPC控制方程：
$$
\boldsymbol{\dot\xi}=f(\boldsymbol{\xi}, \boldsymbol{u}) \tag{1}
$$
其中$\boldsymbol{\xi}=[X_r, Y_r, \varphi]^T$为状态量，$\boldsymbol{u}=[v_r, \delta]^T$ 为控制量。

## Linearization
线性化后的方程为：
$$
\boldsymbol{\dot{\tilde{\xi}}=A\tilde{\xi} + B\tilde{u}} \tag{2}
$$
其中：
$$
A = 
\begin{bmatrix}
	0 & 0 & -v_r\sin\varphi \\
	0 & 0 &  v_r\cos\varphi \\
	0 & 0 & 0
\end{bmatrix}
,\quad
B = 
\begin{bmatrix}
	\cos\varphi & 0 \\
	\sin\varphi & 0 \\
	\frac{\tan\delta}{L} & \frac{v_r}{L\cos^2\delta}
\end{bmatrix}
$$

# Discretization
离散化后的方程：
$$
\boldsymbol{\dot{\tilde{\xi}}=A\tilde{\xi} + B\tilde{u}} \tag{3}
$$
表达式与式（2）一样，但A、B矩阵不同。
$$
\boldsymbol A = 
\begin{bmatrix}
	0 & 0 & -Tv_r\sin\varphi \\
	0 & 0 &  Tv_r\cos\varphi \\
	0 & 0 & 1
\end{bmatrix}
,\quad
\boldsymbol B = 
\begin{bmatrix}
	T\cos\varphi & 0 \\
	T\sin\varphi & 0 \\
	T\frac{\tan\delta}{L} & T\frac{v_r}{L\cos^2\delta}
\end{bmatrix}
$$

## Predict
预测模型：
$$
\boldsymbol{Y}=\boldsymbol{\phi\xi}_{k}+\boldsymbol{\theta U}
$$
其中：
$$
\boldsymbol{Y}=
\begin{bmatrix}
	\boldsymbol{\tilde\xi}_{(k+1)} \\
	\boldsymbol{\tilde\xi}_{(k+2)} \\
	\vdots \\
	\boldsymbol{\tilde\xi}_{(k+Np)} \\
\end{bmatrix}
,\quad
\boldsymbol{\phi}=
\begin{bmatrix}
	\boldsymbol{A}^1 \\
	\boldsymbol{A}^2 \\
	\vdots \\
	\boldsymbol{A}^{Np} \\
\end{bmatrix}
,\quad
\boldsymbol{\theta}=
\begin{bmatrix}
	\boldsymbol{A}^0 & \cdots \\
	\boldsymbol{A}^1 & \boldsymbol{A}^0 & \cdots \\
	\vdots \\
	\boldsymbol{A}^{Np-1} & \cdots & \boldsymbol{A}^{Np-Nc-1} & \boldsymbol{A}^0\\
\end{bmatrix} \boldsymbol{B}
,\quad
\boldsymbol{U}=
\begin{bmatrix}
	\boldsymbol{u}_{k} \\
	\boldsymbol{u}_{(k+1)} \\
	\vdots \\
	\boldsymbol{u}_{(k+Nc)} \\
\end{bmatrix}
$$
$N_p$为预测时域，$N_c$为控制时域。

## Optimization
采用二次规划优化，优化方程为：
$$
\newcommand\j{\boldsymbol J}
\newcommand\x{\boldsymbol X}
\newcommand\h{\boldsymbol H}
\min\j=\frac{1}{2}\x^T\h\x + f^T\x
$$
其中：
$$
\newcommand \j {\boldsymbol J}
\newcommand \h {\boldsymbol H}
\newcommand \e {\boldsymbol E}
\newcommand \q {\boldsymbol Q}
\newcommand \r {\boldsymbol R}
\newcommand \th {\boldsymbol \theta}
\newcommand \ph {\boldsymbol \phi}
\left\{
    \begin{array}{ll}
        \h=2(\th^T\q\th+\r) \\
        f^T=2\ph(\tilde{\xi}_k-\tilde\xi_{ref})\q\th	
    \end{array}
\right.
$$
$Q$、$R$分别为状态量和控制量的权重矩阵。